In [4]:
%matplotlib inline
# Load all necessary packages
import sys
sys.path.append("../")
import numpy as np
from tqdm import tqdm
from warnings import warn
import pandas as pd
from aif360.datasets import StandardDataset
from aif360.metrics import ClassificationMetric, BinaryLabelDatasetMetric
from aif360.algorithms.postprocessing.reject_option_classification\
        import RejectOptionClassification
from common_utils import compute_metrics

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

from IPython.display import Markdown, display
import matplotlib.pyplot as plt
from ipywidgets import interactive, FloatSlider
import pickle

## Huangrui's financial-inclusion-in-africa

In [5]:
def code_continuous(df,collist,Nlevel):
    for col in collist:
        for q in range(1,Nlevel,1):
            threshold = df[~np.isnan(df[col])][col].quantile(float(q)/Nlevel)
            df[col+'_geq'+str(int(q))+'q'+str(threshold)] = (df[col] >= threshold).astype(float)
    df.drop(collist,axis = 1, inplace = True)
    
class Zindi(StandardDataset):
    """financial-inclusion-in-africa dataset.
    """

    def __init__(self, path, label_name='Y', favorable_classes=[1],  
                 protected_attribute_names=['sex'],
                 privileged_classes=[[1]],
                 instance_weights_name=None,
                 categorical_features=[],
                 features_to_drop=[],
                 features_to_keep=[],
                 na_values=[], custom_preprocessing=None,
                 metadata=None):
        """See :obj:`RegressionDataset` for a description of the arguments."""
        

        df = pd.read_csv(path)
        numericals = [col for col in df.columns if len(df[col].unique())>2 and max(df[col])>1]
        code_continuous(df,numericals, 5)

        super(Zindi, self).__init__(
            df=df, label_name=label_name,
            favorable_classes=favorable_classes,
            protected_attribute_names=protected_attribute_names,
            privileged_classes=privileged_classes,
            instance_weights_name=instance_weights_name,
            categorical_features=categorical_features,
            features_to_keep=features_to_keep,
            features_to_drop=features_to_drop, na_values=na_values,
            custom_preprocessing=custom_preprocessing, metadata=metadata)



#### Load dataset and specify options

In [6]:
privileged_groups = [{'sex': 1}]
unprivileged_groups = [{'sex': 0}]
# Metric used (should be one of allowed_metrics)
metric_name = "Equal opportunity difference"
#random seed for calibrated equal odds prediction
random_seed = 12345679
np.random.seed(random_seed)
# Upper and lower bound on the fairness metric used
metric_ub = 0.05
metric_lb = -0.05

In [11]:
experiments_info = {}
budget = 0.01
for K in range(1, 6):
    dataset_orig_train= Zindi(path="./Huangrui/zindi/zindi_train{}.csv".format(K),protected_attribute_names=['sex'],
                privileged_classes=[[1]])
    dataset_orig_test= Zindi(path="./Huangrui/zindi/zindi_test{}.csv".format(K),protected_attribute_names=['sex'],
                privileged_classes=[[1]])
    #only use the budget% of the training data
    dataset_orig_train,_ = dataset_orig_train.split([budget], shuffle=False)

    # Lasso linear classifier and predictions
    X_train = dataset_orig_train.features
    y_train = dataset_orig_train.labels.ravel()
    lmod = pickle.load(open('experiments/zindi'+str(K)+'_sex_bmodel.pkl','rb'))["clf"]
   
    y_train_pred = lmod.predict(X_train)

    dataset_orig_train_pred = dataset_orig_train.copy(deepcopy=True)
    dataset_orig_train_pred.labels = y_train_pred.reshape(-1,1)
    sigmoid = lambda x: 1 / (1 + np.exp(0.5-x))
    dataset_orig_train_pred.scores = sigmoid(y_train_pred).reshape(-1,1)

    dataset_orig_test_pred = dataset_orig_test.copy(deepcopy=True)
    X_test = dataset_orig_test_pred.features
    y_test = dataset_orig_test_pred.labels
    dataset_orig_test_pred.scores = sigmoid(lmod.predict(X_test)).reshape(-1,1)

    #### Best threshold for classification only (no fairness)
    num_thresh = 100
    ba_arr = np.zeros(num_thresh)
    class_thresh_arr = np.linspace(0.01, 0.99, num_thresh)
    for idx, class_thresh in enumerate(class_thresh_arr):
        
        fav_inds = dataset_orig_train_pred.scores > class_thresh
        dataset_orig_train_pred.labels[fav_inds] = dataset_orig_train_pred.favorable_label
        dataset_orig_train_pred.labels[~fav_inds] = dataset_orig_train_pred.unfavorable_label
        
        classified_metric_orig_train = ClassificationMetric(dataset_orig_train,
                                                dataset_orig_train_pred, 
                                                unprivileged_groups=unprivileged_groups,
                                                privileged_groups=privileged_groups)
        
        ba_arr[idx] = 0.5*(classified_metric_orig_train.true_positive_rate()\
                        +classified_metric_orig_train.true_negative_rate())

    best_ind = np.where(ba_arr == np.max(ba_arr))[0][0]
    best_class_thresh = class_thresh_arr[best_ind]

    # print("Best balanced accuracy (no fairness constraints) = %.4f" % np.max(ba_arr))
    # print("Optimal classification threshold (no fairness constraints) = %.4f" % best_class_thresh)
    #### Estimate optimal parameters for the ROC method
    ROC = RejectOptionClassification(unprivileged_groups=unprivileged_groups, 
                                    privileged_groups=privileged_groups, 
                                    low_class_thresh=0.01, high_class_thresh=0.99,
                                    num_class_thresh=100, num_ROC_margin=50,
                                    metric_name=metric_name,
                                    metric_ub=metric_ub, metric_lb=metric_lb)
    ROC = ROC.fit(dataset_orig_train, dataset_orig_train_pred)
    # print("Optimal classification threshold (with fairness constraints) = %.4f" % ROC.classification_threshold)
    # print("Optimal ROC margin = %.4f" % ROC.ROC_margin)

    # Metrics for the train set
    fav_inds = dataset_orig_train_pred.scores > best_class_thresh
    dataset_orig_train_pred.labels[fav_inds] = dataset_orig_train_pred.favorable_label
    dataset_orig_train_pred.labels[~fav_inds] = dataset_orig_train_pred.unfavorable_label

    # display(Markdown("#### train set"))
    # display(Markdown("##### Raw predictions - No fairness constraints, only maximizing balanced accuracy"))

    metric_train_bef = compute_metrics(dataset_orig_train, dataset_orig_train_pred, 
                    unprivileged_groups, privileged_groups)

    # Transform the validation set
    dataset_transf_train_pred = ROC.predict(dataset_orig_train_pred)

    # display(Markdown("#### train set"))
    # display(Markdown("##### Transformed predictions - With fairness constraints"))
    metric_train_aft = compute_metrics(dataset_orig_train, dataset_transf_train_pred, 
                    unprivileged_groups, privileged_groups)
    # Testing: Check if the metric optimized has not become worse
    assert np.abs(metric_train_aft[metric_name]) <= np.abs(metric_train_bef[metric_name])

    # Metrics for the test set
    fav_inds = dataset_orig_test_pred.scores > best_class_thresh
    dataset_orig_test_pred.labels[fav_inds] = dataset_orig_test_pred.favorable_label
    dataset_orig_test_pred.labels[~fav_inds] = dataset_orig_test_pred.unfavorable_label

    # display(Markdown("#### Test set"))
    # display(Markdown("##### Raw predictions - No fairness constraints, only maximizing balanced accuracy"))

    metric_test_bef = compute_metrics(dataset_orig_test, dataset_orig_test_pred, 
                    unprivileged_groups, privileged_groups)
    # Metrics for the transformed test set
    dataset_transf_test_pred = ROC.predict(dataset_orig_test_pred)

    # display(Markdown("#### Test set"))
    # display(Markdown("##### Transformed predictions - With fairness constraints"))
    metric_test_aft = compute_metrics(dataset_orig_test, dataset_transf_test_pred, 
                    unprivileged_groups, privileged_groups)

    #自己计算error, 不是balanced accuracy！！！
    print("K = {}, budget = {}".format(K, budget))
    print("The Error for the test dataset is {:.4}".format(np.mean(dataset_orig_test.labels!=dataset_transf_test_pred.labels)))
    print("The Equal opportunity difference for the test dataset is {:.4}".format(metric_test_aft["Equal opportunity difference"]))
    experiments_info["K = {}, budget = {}".format(K, budget)] = {"Error": np.mean(dataset_orig_test.labels!=dataset_transf_test_pred.labels), "Equal opportunity difference": metric_test_aft["Equal opportunity difference"]}

Balanced accuracy = 0.8054
Statistical parity difference = -0.2974
Disparate impact = 0.3264
Average odds difference = -0.1831
Equal opportunity difference = -0.1333
Theil index = 0.0742
Balanced accuracy = 0.8295
Statistical parity difference = -0.1744
Disparate impact = 0.5203
Average odds difference = -0.0625
Equal opportunity difference = -0.0333
Theil index = 0.0666
Balanced accuracy = 0.7598
Statistical parity difference = -0.2122
Disparate impact = 0.4009
Average odds difference = -0.1881
Equal opportunity difference = -0.2226
Theil index = 0.0868
Balanced accuracy = 0.7464
Statistical parity difference = -0.1053
Disparate impact = 0.6366
Average odds difference = -0.0546
Equal opportunity difference = -0.0508
Theil index = 0.0909
K = 1, budget = 0.01
The Error for the test dataset is 0.1854
The Equal opportunity difference for the test dataset is -0.05078
Balanced accuracy = 0.7580
Statistical parity difference = -0.2471
Disparate impact = 0.3778
Average odds difference = -0.20

In [12]:
experiments_info

{'K = 1, budget = 0.01': {'Error': 0.18537414965986396,
  'Equal opportunity difference': -0.050778543558326894},
 'K = 2, budget = 0.01': {'Error': 0.4871413390010627,
  'Equal opportunity difference': -0.07591075554581406},
 'K = 3, budget = 0.01': {'Error': 0.6567481402763018,
  'Equal opportunity difference': -0.03347288333237752},
 'K = 4, budget = 0.01': {'Error': 0.4172157279489904,
  'Equal opportunity difference': -0.20398388809843482},
 'K = 5, budget = 0.01': {'Error': 0.4412327311370882,
  'Equal opportunity difference': -0.09080234833659495}}